# Agents 🤖

Agents are like "tools" for LLMs. They allow a LLM to access Google search, perform complex calculations with Python, and even make SQL queries.

In this notebook we'll explore agents and how to use them in LangChain.

We'll start by installing the prerequisite libraries that we'll be using in this example.

In [ ]:
!pip install -qU langchain openai google-search-results wikipedia

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted.

In [ ]:
OPENAI_API_KEY = "xxx"
SERPAPI_KEY = 'xxx'

In [ ]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

As we did before, we will be counting our tokens in each call.

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

With all of that set up, let's jump into **Agents**.

## What is an agent?

**Definition**: The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:


> Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In this edition we will cover what we may call 'generic' agents which really able to perform many meta tasks. There are other more specific agents that are tuned for different tasks (called 'agent-toolkits'), but we will cover those in a future edition.

## Agent types

In this section we will review several agents and see how they 'think' and what they can do.

Using one of langchain's pre-built agents involves three variables:
* defining the tools or the toolkit
* defining the llm
* defining the agent type

This is all really easy to do in langchain, as we will see in the following example.

### Agent type #1: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

We will use the math tool in this example and load it as below:

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

The memory type being used here is a simple buffer memory to allow us to remember previous steps in the reasoning chain.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent_types import AgentType

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [ ]:
result = count_tokens(
    conversational_agent,
    "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?"
)

As we can see below, the prompt is similar but it includes a great prelude of instructions that make it an effective assistant as well + a spot for including the chat history from the memory component:

In [ ]:
print(conversational_agent.agent.llm_chain.prompt.template)

Let's see what happens if we try to answer the question that is related to the previous one:

In [ ]:
result = count_tokens(
    conversational_agent,
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)

### Agent type #2: React Docstore

This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have two and only two tools at its disposal: 'Search' and 'Lookup'.

With 'Search' it will bring up a relevant article and with 'Lookup' the agent will find the right piece of information in the article. This is probably easiest to see in an example:

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [ ]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [ ]:
count_tokens(docstore_agent, "How high is the Stuttgart Fernsehturm in meters?")

The prompt to this agent contains several examples of the `Question` > `Thought` > `Action` > `Observation` loop, that include the `Search` and `Lookup` tools.

If you want to learn more about this approach [this](https://arxiv.org/pdf/2210.03629.pdf) is the paper for ReAct

In [ ]:
print(docstore_agent.agent.llm_chain.prompt.template)

### Agent type #3: Self Ask with Search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [ ]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key=SERPAPI_KEY)
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

We will not interact with this agent because for that we would need a serpapi key. However, by checking out the prompt we can see a few examples of how it works:

In [ ]:
count_tokens(self_ask_with_search, "What is the profession of Prof. Markus Reischl?")

As we can see, the prompt is basically a series of many examples to show the LLM how to ask follow up questions to a search tool until it can get to the final answer.

# Outlook
This is just a very quick introduction into using LLMs with LangChain and the OpenAI API. We merely scratched the surface of whats possible. If you want to continue learning about the practical applications of LLMs i highly recommend the following ressoucres:

1. The Youtube series [LangChain for Gen AI and LLMs](https://www.youtube.com/watch?v=nE2skSRWTTs&list=PLIUOU7oqGTLieV9uTIFMm6_4PXg-hlN6F) of James Briggs.
2. The [LangChain Handbook Code](https://github.com/pinecone-io/examples/tree/master/learn/generation/langchain/handbook
) (of which this tutorial is based on).
3. The [LangChain documentation](https://python.langchain.com/docs/get_started/introduction) (duh!)
4. The [LangChain Hub](https://smith.langchain.com/o/dddd8041-57d1-56c4-a1c9-aa67b55323c1) containing many prompts, chains, agents, and more (still in closed beta).